In [10]:
with open('collection.txt', 'r') as collection_file:
    x = collection_file.readlines()
x = list(filter(lambda x: x != '\n', x))
x = list(map(lambda x: x[:-2], x))
facts = [['Калифорнийский посёлок (на илл.) был независим от США почти 3 месяца, пока не пришла пора праздновать День независимости и закупать алкоголь.'],
         ['В своём театре пантомимы польский хореограф ставил «Фауста».'],
         ['Конфликт авторов «Песни о друге» чуть было не привёл к мордобою.']]

In [11]:
import re
import numpy as np
import pymorphy2


MORPH = pymorphy2.MorphAnalyzer()

#-----------------------------------------------------------
re_tokenizer = re.compile(r'[\w]+')
re_stopwordy = re.compile(r'[\d_]+|[\d]{2,}[\w]+')
morph_cache = {}


def tokenize(s):
    global morph_cache

    s = s.lower().replace('ё', 'е')
    tokens = []
    for word in re_tokenizer.findall(s):
        if re_stopwordy.match(word):
            continue

        wn = morph_cache.get(word, None)
        
        if wn == None:
            p = MORPH.parse(word)[0]
            wn = p.normal_form
                
            morph_cache[word] = wn

        tokens.append(wn)

    return tokens

In [12]:
class Vectorizer:
    def __init__(self, collection):
        self.df_dict = {}
        self.idf_dict = {}
        self.collection_len = len(collection)
        # проход для составления словаря вида "term: df(term)"
        for doc in collection:
            tokens = tokenize(doc)
            for token in tokens:
                if token in self.df_dict.keys():
                    self.df_dict[token] += 1
                else:
                    self.df_dict[token] = 0
        # проход для составления словаря вида "term: idf(term)"
        for token in self.df_dict:
            self.idf_dict[token] = np.log((self.collection_len + 1) / (self.df_dict[token] + 1))
    def vectorize(self, collection, idf=False):
        vecs = []
        for doc in collection:
            vec = []
            tokens = tokenize(doc)
            for token in self.df_dict:
                counter = len(list(filter(lambda x: x==token, tokens)))
                if idf:
                    counter *= self.idf_dict[token]
                vec.append(counter)
            vecs.append(vec / np.linalg.norm(vec))
        return vecs


In [17]:
# получаем наиболее релевантные предложения по векторной модели без idf
from sklearn.metrics.pairwise import cosine_similarity


vec_text = Vectorizer(x)
X = vec_text.vectorize(x)

for fact in facts:
    print(fact[0])
    fact = vec_text.vectorize(list(fact))
    Y = list(map(lambda x: x[0], cosine_similarity(X, fact)))
    print('---------------')
    k = 1
    for i in sorted(set(sorted(Y, reverse=True)),reverse=True):
        for j in np.where(Y == i)[0]:
            print(str(k)+'.', x[j])
            k += 1
    print('---------------')

Калифорнийский посёлок (на илл.) был независим от США почти 3 месяца, пока не пришла пора праздновать День независимости и закупать алкоголь.
---------------
1. 7 апреля 1850 года калифорнийский посёлок золотоискателей Раф-энд-Рэди объявил о своём отделении от США и стал «независимым государством».
2. День независимости США (англ. Independence Day) — день принятия Декларации независимости США в 1776 году, которая провозглашает независимость США от Королевства Великобритании; празднуется в Соединенных Штатах Америки 4 июля.
3. День Независимости является федеральным праздником, поэтому все не очень важные федеральные институты (такие как Почтовая служба США и Федеральный суд США) не работают в этот день.
4. С самого начала, американцы праздновали независимость 4 июля, в день, когда была провозглашена Декларация Независимости.
5. День независимости — это национальный праздник США
6. Убив в драке Валентина, спутники покидают город, и Фауст не вспоминает Маргариту до тех пор, пока не встре

251. 1961 — премия 2-го Вроцлавского театрального фестиваля за ценный художественный поиск (спектакль «Кабинет диковинок»
252. «Фа́уст, траге́дия» (нем. Faust. Eine Tragödie.), чаще просто «Фа́уст» — философская драма для чтения, которая считается главным трудом Иоганна Вольфганга Гёте.
253. Содержит наиболее известный вариант легенды о докторе Фаусте
254. Над второй частью (нем. Faust: der Tragödie zweiter Teil) Гёте трудился в преклонные годы; она увидела свет уже после его смерти, в 1832 году.
255. Осознавая, что сокровенные тайны мироздания недоступны человеческому разуму, в отчаянии он подносит к губам склянку с ядом.
256. Лишь внезапно зазвучавший благовест предотвращает самоубийство
257. Бродя по городу со своим учеником Вагнером, Фауст встречает собаку, которую приводит за собой в дом, где она принимает человеческий образ Мефистофеля.
258. Злой дух после ряда искушений убеждает старого отшельника вновь изведать радости опостылевшей ему жизни.
259. Плата за это — душа Фауста.
26

In [14]:
# получаем наиболее релевантные предложения по векторной модели с idf
from sklearn.metrics.pairwise import cosine_similarity


vec_text = Vectorizer(x)
X = vec_text.vectorize(x, idf=True)

for fact in facts:
    print(fact[0])
    fact = vec_text.vectorize(list(fact), idf=True)
    Y = list(map(lambda x: x[0], cosine_similarity(X, fact)))
    print('---------------')
    k = 1
    for i in sorted(set(sorted(Y, reverse=True)),reverse=True):
        for j in np.where(Y == i)[0]:
            print(str(k)+'.', x[j])
            k += 1
    print('---------------')

Калифорнийский посёлок (на илл.) был независим от США почти 3 месяца, пока не пришла пора праздновать День независимости и закупать алкоголь.
---------------
1. 7 апреля 1850 года калифорнийский посёлок золотоискателей Раф-энд-Рэди объявил о своём отделении от США и стал «независимым государством».
2. Убив в драке Валентина, спутники покидают город, и Фауст не вспоминает Маргариту до тех пор, пока не встречает её призрака на шабаше.
3. Оно не работало несколько месяцев в 1855 году, несколько месяцев в 1913 году и с 1942 по 1948 год.
4. С самого начала, американцы праздновали независимость 4 июля, в день, когда была провозглашена Декларация Независимости.
5. Независимое государство с 7 апреля по 4 июля 1850 года.
6. День независимости США (англ. Independence Day) — день принятия Декларации независимости США в 1776 году, которая провозглашает независимость США от Королевства Великобритании; празднуется в Соединенных Штатах Америки 4 июля.
7. Почтовая служба США требовала от жителей, для 

159. Некоторые песни поют только на определённых территориях, например, «Янки-дудл» в северо-восточных штатах или «Дикси» в южных штатах.
160. В 1954—1958 годах — редактор Ленинградского отделения издательства «Музыка»
161. Над второй частью (нем. Faust: der Tragödie zweiter Teil) Гёте трудился в преклонные годы; она увидела свет уже после его смерти, в 1832 году.
162. День независимости США (англ. Independence Day) — день принятия Декларации независимости США в 1776 году, которая провозглашает независимость США от Королевства Великобритании; празднуется в Соединенных Штатах Америки 4 июля.
163. Этот балет поставлен на десятках отечественных и зарубежных сцен, а оркестровые фрагменты из него, объединённые автором в сюиты, вошли в постоянный филармонический репертуар
164. Второй день июля 1776 года станет самым незабываемым в истории Америки.
165. «Песня о друге» («Если радость на всех одна…») — популярная песня композитора Андрея Петрова на стихи Григория Поженяна, написанная для киноф